<a href="https://colab.research.google.com/github/Victor-A-Orlov/Test/blob/master/test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

## Импортируем необходимые библиотеки

In [12]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Загружаем данные
Атрибут data содержит каждое изображение цифры в виде вектора длиной 64, что нам и нужно

In [13]:
digits = load_digits()
X = digits.data
y = digits.target
print(X)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]


Отложим 25% для контрольной выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Создадим классификатор на основе метода одного ближайшего соседа (nearest neighbour). При этом для поиска ближайшего соседа вместо простого перебора используем структуру данных "k-d tree", как предлагается в задании.

## Поиск ближайшего соседа с помощью **k-d tree**

Для начала создадим класс `Tree`, создающий двоичное дерево, последовательно добавляя новые значения с помощью метода `insert`, и класс `Node` узла этого дерева. 

In [15]:
class Node:
    def __init__(self, X, y, alignment_axis=0):
        # Левые и правые "ветви" узла:
        self.left = None
        self.right = None
        # Ось по которой будет происходить разделение:
        self.axis = 0
        # Данные в узле:
        self.X = X
        self.y = y
        
    # Для удобства вывода содержимого узла
    def __call__(self):
        return self.data

In [16]:
class Tree:
    def __init__(self, val=None):
        self.root = None
        
    def insert(self, root=None, node=None):
        if self.root is None:
            self.root = node
        else:
            if root is None:
                root = self.root
            
            dim = node.X.shape[0]
            # Здесь мы циклично перебираем оси в пространства X (в задании их 64). 
            # Можно сделать также случайный перебор
            node.axis = (root.axis + 1) % dim
            if node.X[root.axis] < root.X[root.axis]:
                if root.left is None:
                    root.left = node
                else:
                    self.insert(root.left, node)
            else:
                if root.right is None:
                    root.right = node
                else:
                    self.insert(root.right, node)

Теперь нам нужна функция чтобы найти ближайшего соседа для искомой точки и для массива точек

In [17]:
def find_one_neighbor(start_node, query):
    best_distance = float("inf")
    best_node = start_node
    node = start_node
    while True:
        # Евклидово расстояние:
        dist = np.linalg.norm(node.X - query)
        if dist < best_distance:
            best_distance = dist
            best_node = node
        if query[node.axis] < node.X[node.axis]:
            node = node.left
            if node is None:
                break
        else:
            node = node.right
            if node is None:
                break
    
    return best_node

Наконец, сделаем класс классификатора, который обучается на данных и предсказывает классы искомых точек.

In [18]:
def query_targets(start_node, query):
    results = []
    for q in query:
        node = find_one_neighbor(start_node, q)
        results.append(node.y)
    return np.array(results)

class NearestNeighborClassfier :
    def __init__(self) -> None:
        self.tree = Tree()

    def fit(self, X, y):
        for x_sample, y_sample in zip(X, y):
            self.tree.insert(node=Node(x_sample, y_sample))

    def predict(self, query):
        return query_targets(self.tree.root, query)

## Результаты

Проверим точность (accuracy) классификатора на тестовой выборке:

In [19]:
clf = NearestNeighborClassfier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.88


# Задание 2

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9755555555555555
